In [1]:
import pandas as pd

sample1 = pd.read_csv('data/output.csv', names=["book1"])  #index_col=0 하면 안됨
sample1.sample(n=8)

,book1
61,잇수로 치면 여기서 거기가 시오 리가 넘는답니다.
222,만일 김첨지가주기를 띠지 않았던들 한 발을 대문에 들여놓았을 제 그곳을 지배하는 ...
187,“갔다가 그저 오기가 안됐데그려.
106,"김첨지는 구걸하는 거지나 무엇같이 연해연방 그의 기색을 살피며, “아씨, 정거장..."
217,김첨지는 화증을 내며 확신 있게 소리를 질렀으되 그 소리엔 안 죽은 것을믿으려고 ...
158,"“아따 이놈아, 사십 전이 그리 끔찍하냐."
33,아마도 그 학교 기숙사에 있는 이로 동기방학을 이용하여 귀향하려 함이리라.
143,"“아따, 재미 안 좋다고 술 못 먹을 낸가."


In [1]:
import matplotlib.pyplot as plt

ImportError: DLL load failed while importing _imaging: 지정된 모듈을 찾을 수 없습니다.

In [2]:
sample1.columns

Index(['book1'], dtype='object')

In [3]:
# 라이브러리 불러오기
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook
from sklearn.model_selection import train_test_split
 
#kobert
from kobert_tokenizer import KoBERTTokenizer

# transformers
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

# GPU 사용시 필요
device = torch.device("cuda:0")

In [4]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')
tok = tokenizer.tokenize

In [5]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6, #클래스 수 조정
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [6]:
model = torch.load('model/kobert-v3.pt')
model.eval()

BERTClassifier(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(8002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True

In [7]:
# Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5  # 에포크 횟수
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer,vocab, max_len,
                 pad, pair):
   
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))
         
    def __len__(self):
        return (len(self.labels))

def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=2)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer
     

In [8]:
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [9]:
emos = ['행복','불안','놀람', '슬픔','분노','중립']
def sa_demo(sentence):
    res = predict(sentence)
    print("문장에서 "+ emos[res] + "이(가) 느껴집니다.")

In [10]:
emos = ('행복','불안','놀람', '슬픔','분노','중립')
res = []
for index, data in sample1.iterrows():
    res.append(emos[predict(data['book1'])])
res

['행복',
 '중립',
 '행복',
 '중립',
 '중립',
 '행복',
 '행복',
 '행복',
 '분노',
 '슬픔',
 '중립',
 '불안',
 '불안',
 '분노',
 '분노',
 '분노',
 '분노',
 '중립',
 '중립',
 '슬픔',
 '분노',
 '분노',
 '분노',
 '중립',
 '중립',
 '중립',
 '중립',
 '중립',
 '놀람',
 '중립',
 '중립',
 '중립',
 '불안',
 '중립',
 '놀람',
 '중립',
 '중립',
 '분노',
 '불안',
 '놀람',
 '슬픔',
 '중립',
 '슬픔',
 '중립',
 '분노',
 '분노',
 '중립',
 '중립',
 '중립',
 '중립',
 '중립',
 '분노',
 '중립',
 '불안',
 '놀람',
 '분노',
 '중립',
 '중립',
 '중립',
 '놀람',
 '중립',
 '중립',
 '중립',
 '행복',
 '중립',
 '분노',
 '놀람',
 '중립',
 '중립',
 '중립',
 '놀람',
 '중립',
 '슬픔',
 '중립',
 '슬픔',
 '분노',
 '중립',
 '불안',
 '중립',
 '중립',
 '중립',
 '중립',
 '중립',
 '행복',
 '슬픔',
 '행복',
 '행복',
 '슬픔',
 '중립',
 '중립',
 '슬픔',
 '슬픔',
 '중립',
 '중립',
 '분노',
 '분노',
 '중립',
 '분노',
 '행복',
 '불안',
 '중립',
 '불안',
 '중립',
 '중립',
 '중립',
 '중립',
 '중립',
 '중립',
 '분노',
 '중립',
 '중립',
 '슬픔',
 '중립',
 '중립',
 '슬픔',
 '중립',
 '중립',
 '중립',
 '중립',
 '분노',
 '불안',
 '불안',
 '중립',
 '중립',
 '슬픔',
 '중립',
 '슬픔',
 '중립',
 '슬픔',
 '중립',
 '중립',
 '중립',
 '중립',
 '중립',
 '중립',
 '중립',
 '분노',
 '중립',
 '행복',
 '행복',
 '행복',
 '중립',
 '중립',

In [ ]:
model = torch.load('model/kobert-v3.pt')
model.eval()

In [ ]:
def predict(sentence):
    dataset = [[sentence, '0']]
    test = BERTDataset(dataset, 0, 1, tok, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(test, batch_size=batch_size, num_workers=0)
    model.eval()
    answer = 0
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        for logits in out:
            logits = logits.detach().cpu().numpy()
            answer = np.argmax(logits)
    return answer

In [12]:
emos = ('행복','불안','놀람', '슬픔','분노','중립')
res = {'행복':0,'불안':0,'놀람':0, '슬픔':0,'분노':0,'중립':0}

for index, data in sample1.iterrows():
    res[emos[predict(data['book1'])]] += 1
res

{'행복': 19, '불안': 15, '놀람': 15, '슬픔': 26, '분노': 48, '중립': 133}

In [11]:
from collections import Counter
counter = Counter(res)


for emotion, count in counter.items():
    print(f"{emotion}: {count}")

행복: 19
중립: 133
분노: 48
슬픔: 26
불안: 15
놀람: 15


In [1]:
import matplotlib.pyplot as plt

# 딕셔너리 생성
my_dict = {"행복": 5, "중립": 10, "분노": 7, "슬픔": 3, "불안": 2}

# 키와 값 리스트 추출
labels = list(my_dict.keys())
sizes = list(my_dict.values())

# 원형 그래프 그리기
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90)
ax1.axis('equal')  # 원형을 유지하도록 축 조정

plt.show()

ModuleNotFoundError: No module named 'matplotlib.pyplot'